In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from sympy import pretty_print
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

### load files

In [2]:
loader = DirectoryLoader("./files_used")
docs = loader.load()


### Embedding

In [3]:
# Embeddings and vector store
import shutil
from langchain.vectorstores import Chroma


# from langchain_community.embeddings.sentence_transformer import (
#     SentenceTransformerEmbeddings,
# )

# persist_directory = './db/chromas-e5-mistral'
# persist_directory = './db/chromas-camembert'
persist_directory = './db/chromas-e5-large'

In [4]:
import os 

if os.path.exists("model_files/e5-large/multilingual-e5-large"):
    print("Model already exists")
else:
    print("Downloading model")

Model already exists


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
# modelPath = "model_files/dangvantuan/sentence-camembert-large"
# modelPath = "model_files/Salesforce/SFR-Embedding-Mistral"
# modelPath = "model_files/mistral-e5/e5-mistral-7b-instruct"
modelPath = "model_files/e5-large/multilingual-e5-large"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [6]:
def all_splitter(docs):
    # Markdown 
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
        md_docs.extend(md_doc)
    

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400, chunk_overlap=200, add_start_index=True, separators=['\n\n', '\n', '(?<=\. )']
    )
    all_splits = text_splitter.split_documents(md_docs)
    return all_splits

def markdown_splitter(docs):
    # Markdown 
    headers_to_split_on = [
        ("#", "Titre 1"),
        ("##", "Sous-titre 1"),
        ("###", "Sous-titre 2"),
    ]
    
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    # Split based on markdown and add original metadata
    md_docs = []
    for doc in docs:
        md_doc = markdown_splitter.split_text(doc.page_content)
        for i in range(len(md_doc)):
            md_doc[i].metadata = md_doc[i].metadata | doc.metadata 
        md_docs.extend(md_doc)
    
    return md_docs


In [7]:
chunks_3 = markdown_splitter(docs)

In [9]:
chunks_3

[Document(page_content="Gan Assurances s'engage pour soutenir les salariés aidants . Aujourd’hui, on compte 11 millions d’aidants. D’ici 2030, 1 actif sur 4 aura, en plus de son emploi, à s’occuper d’un proche. L'«aidant» est défini par la loi depuis 2015 comme une : « personne qui vient en aide, de manière régulière et fréquente, à titre non professionnel, pour accomplir tout ou partie des actes ou des activités de la vie quotidienne d’une personne en perte d’autonomie, du fait de l’âge, de la maladie ou d’un handicap. » Conscient de ces enjeux depuis plusieurs années, Gan Assurances met en place des mesures destinées à favoriser la conciliation de la vie professionnelle et de la vie familiale de ces salariés aidants. Ces dispositifs résultent de l’accord UES relatif aux salariés en situation de handicap ou accompagnant un membre de leur famille touché par la maladie ou le handicap du 20 novembre 2020. Est également concerné, l’Accord relatif à la Diversité et à l’Egalité des chances 

In [ ]:
# If the directory exists, first delete it
try:
    shutil.rmtree(persist_directory)
except FileNotFoundError as e:
    pass

# Create vector store and save the db
db = Chroma.from_documents(
    chunks_3, 
    embeddings,
    persist_directory=persist_directory
)
db.persist()

In [ ]:
db._collection.count()

### LLM Model

In [4]:
import torch

torch.cuda.is_available()

True

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline


MODEL_NAME = "model_files/mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True,local_files_only=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float32,
    trust_remote_code=True,
    device_map="auto",
    local_files_only=True,
    # quantization_config=quantization_config
)

In [ ]:


generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    max_new_tokens=1048,
)

In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

### Chroma connection

In [2]:
# Embeddings and vector store
import shutil
from langchain.vectorstores import Chroma

persist_directory = './db/chromas-e5-large'

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define the path to the pre-trained model you want to use
# modelPath = "model_files/dangvantuan/sentence-camembert-large"
# modelPath = "model_files/Salesforce/SFR-Embedding-Mistral"
# modelPath = "model_files/mistral-e5/e5-mistral-7b-instruct"
modelPath = "model_files/e5-large/multilingual-e5-large"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [5]:
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [6]:
from transformers import pipeline

qa_model = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")
qa_model(question = question, context = context)

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """
### [INST] 
Instruction: Vous êtes un assistant des ressources humaine au sein de la compagnie d'assurance de Gan Assurances. Des questions et des données de contexte vous seront fournies. Répondez aux questions en utilisant les données de contexte fournies. Si la réponse n'est pas dans le contexte ou que le contexte ne correspond pas à la question, répondez simplement par "Je ne sais pas".
De plus, vous répondrez en français.

contexte : {context} 


### Question : {question}

[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 4, 
    }
)



retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9}
)

# retriever = db.as_retriever(
#     search_type="mmr"
# )

chain_type_kwargs={
        "prompt": prompt
    }

vector_dbqa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True)

In [12]:
retriever_test = retriever.get_relevant_documents("Combien ais-je de jours de congé par an ?", consider_metadata=True)

In [14]:
retriever_test

[Document(page_content="Vous pouvez bénéficier de jours d'absences autorisées à caractère exceptionnel, pour les motifs ci-dessous. Permis de conduire \u200b\u200b\u200b\u200b\u200b\u200b\u200bIl est possible de prendre 1/2 journée de congé en vue de l'examen pour l'obtention du permis de conduire. Déménagement Les salariés titulaires peuvent prendre 1 journée de congé au titre d'un déménagement (une fois par an).", metadata={'Sous-titre 1': 'Autres congés :', 'Titre 1': 'Congés enfants', 'source': 'files_used\\enfants.txt'}),
 Document(page_content='La durée des congés payés pouvant être pris en une seule fois ne peut excéder 20 jours ouvrés. Le congé principal (congés d’été) doit être pris du 1er mai au 31 octobre pour une durée minimum de 10 jours ouvrés consécutifs et ne dépassant pas 20 jours ouvrés. Il est cependant recommandé que les congés d’été soient pris, dans la mesure du possible, entre le 1er juin et le 30 septembre. Les autres jours peuvent être pris en dehors de la péri

In [17]:
context = ""
for i in range(len(retriever_test)):
    context += retriever_test[i].page_content + "\n"

In [16]:
qa_model(question = "Combien ais-je de jours de congé par an ?", context = context)

{'score': 1.5953102092680638e-06,
 'start': 0,
 'end': 36,
 'answer': 'My name is Tim and I live in Sweden.'}

In [ ]:
vector_dbqa_chain("Combien ais-je de jours de congé par an ?")

In [8]:
import pandas as pd
Q = pd.read_excel("test-mistral-camembert-RH.xlsx")

In [ ]:
Q=Q.head(15)

In [ ]:
from tqdm import tqdm
import time

Q["result"] = ""
Q["time"] = ""
for i in tqdm(Q.index):
    start_time = time.time()
    question = Q["Q"][i]
    Q["result"][i] = retriever.get_relevant_documents(question, consider_metadata=True)
    Q["time"][i] = time.time() - start_time
    Q["answer"][i] = qa_model(question = question, context = context)



In [ ]:
Q["result"][0][0].page_content

In [ ]:
Q["réponse"] = ""
Q["documents"] = ""
for i in Q.index:
    compt = 0
    for j in range(len(Q["result"][i])):
        Q["documents"][i] += str(compt) + " : " + Q["result"][i][j].page_content +" \n "
        compt+=1

In [ ]:
Q.to_excel("test-e5miltiligual-RH.xlsx")

In [ ]:
retriever.invoke("Combien ais-je de jours de artt ?")

In [ ]:
Q.to_excel("test-camembert-RH-result-chunk800.xlsx")

In [ ]:
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)



In [ ]:
retriever.get_relevant_documents("Combien ais-je de jour de congés ?", consider_metadata=True)

In [ ]:
def ask(question):
   context = retriever.get_relevant_documents(question, consider_metadata=True)
   print(context)

   answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
   return answer

In [ ]:
user_question = "Comment voir mes congés ?"
answer = ask(user_question)
print("Answer:", answer)

In [ ]:
from tqdm import tqdm
import time

Q["result"] = ""
Q["time"] = ""
for i in tqdm(Q.index):
    start_time = time.time()
    question = Q["Q"][i]
    answer = ask(question)
    Q["result"][i] = answer
    Q["time"][i] = time.time() - start_time



In [ ]:
Q.to_excel("test-mistral-camembert-RH-result-2.xlsx")